<p><font size="5"><b>PROJET 2 - SIMPLON DEV. DATA : PYTHON - SQL </b></font></p>


<p><font size="4"><b> *0-2. Scrapping d’un site internet à l’aide de beautiful soup - data pre-processing*  
 - 02/01/2010* </b></font></p>

<p><font size="3"> *Scraping du site internet www.paruvendu.fr - liste de toutes les annonces de ventes de biens
    immobiliers ancien PRO scrappées le 29/01/2020* </font></p>

*© 2020, Gérald Bouget  (<mailto:geraldbouget@gmail.com>)

___

In [1]:
import pandas as pd
import numpy as np
import re
import concurrent.futures

In [2]:
df = pd.read_csv('paruvendu_ancien_pro.csv')

In [418]:
df.head()

,Unnamed: 0,titre_annonce,ville_cp,prix,texte_annonce,nom_agence
0,0,"Appartement 2 pièces, 33 m² Paris 18",Paris 18,420 000 €,Place Saint Pierre / Butte Montmartrre. 2 piè...,DISTRICT GEORGE V
1,1,"Appartement 5 pièces, 146 m² Paris 8",Paris 8,2 660 000 €,TRIANGLE d'OR proche de l'avenue Montaigne - ...,DISTRICT GEORGE V
2,2,"Maison 5 pièces, 65 m² Athis-Mons (91200)",Athis-Mons (91200),334 000 €,PLATEAU JOLIE MEULIERE 120 M² CALME NON MITOY...,Réseau Swixim
3,3,"Appartement 3 pièces, 70 m² carrez Bondoufle ...",Bondoufle (91070),202 800 €,VIEUX CENTRE - Appartement 3 pièces - 70m² Si...,
4,4,"Maison 3 pièces, 66 m² La Ferté-sous-Jouarre ...",La Ferté-sous-Jouarre (77260),95 500 €,"Maison de hameau avec courette, vue dégagée A...",AGENCE DE LA VALLEE


In [419]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17600 entries, 0 to 17599
Data columns (total 6 columns):
Unnamed: 0       17600 non-null int64
titre_annonce    17600 non-null object
ville_cp         17600 non-null object
prix             17600 non-null object
texte_annonce    17600 non-null object
nom_agence       17600 non-null object
dtypes: int64(1), object(5)
memory usage: 825.1+ KB


# Vérification duplicated rows

In [420]:

df[df.duplicated()]

,Unnamed: 0,titre_annonce,ville_cp,prix,texte_annonce,nom_agence


# Suppression des lignes où il y a absence de prix

In [421]:

df = df[df['prix']!=' ']

In [422]:
df[df['prix']==' ']

,Unnamed: 0,titre_annonce,ville_cp,prix,texte_annonce,nom_agence


# Création variable 'type de bien'

In [423]:

df['type_bien']= ''

In [424]:
df.titre_annonce = df.titre_annonce.astype(str)

### Extraction du type d'appartement dans colonne titre_annonce et insertion du type de bien dans colonne type de bien

In [425]:

liste_type_bien=['Appartement', 'Maison', 'Chalet', 'Loft', 'château', 'Villa', 'Hôtel Particulier',
                 'Duplex/triplex', 'Chambre', 'Ferme', 'Atelier', 'Plateau', 'Moulin']

for i in range(len(df.index)):
    for tb in liste_type_bien:            
        if df.iloc[i, 1].__contains__(tb)==True:
            df.iloc[i, 6] = tb
        else:
            pass #obligé de mettre ça. Une autre valeur créé un un affichage erronné dans colonne type de bien
 

In [426]:
df[df['type_bien']==''] # vérification que toutes les cellules ont une valeur

,Unnamed: 0,titre_annonce,ville_cp,prix,texte_annonce,nom_agence,type_bien


# Création d'une colonne code postal

In [427]:

cp_reg = re.compile(r'\((\w){5}\)') #pattern pour code postal incluant parenthese 
df['code_postal'] = ''


### Extraction cp dans colonne cp_ville et insertion dans colonne code_postal

In [428]:

for i in range(len(df.index)):
        try:            
            df.iloc[i, 7] = re.sub(r'[\(\)]','',cp_reg.search(df.iloc[i, 2]).group())
        except (AttributeError):
            pass
        else:
            df.iloc[i, 7] = re.sub(r'[\(\)]','',cp_reg.search(df.iloc[i, 2]).group())
        

### Vérification des valeurs nulles dans colonne CP

In [429]:

#df[(df.code_postal == '') & (df.ville_cp.str.contains('Paris'))]
df[df.code_postal == '']

,Unnamed: 0,titre_annonce,ville_cp,prix,texte_annonce,nom_agence,type_bien,code_postal
0,0,"Appartement 2 pièces, 33 m² Paris 18",Paris 18,420 000 €,Place Saint Pierre / Butte Montmartrre. 2 piè...,DISTRICT GEORGE V,Appartement,
1,1,"Appartement 5 pièces, 146 m² Paris 8",Paris 8,2 660 000 €,TRIANGLE d'OR proche de l'avenue Montaigne - ...,DISTRICT GEORGE V,Appartement,
5,5,"Appartement 2 pièces, 47 m² Paris 15",Paris 15,415 000 €,Appartement 2 pièces 47m2 à Paris en résidenc...,EXPERTIMO,Appartement,
27,27,"Maison 9 pièces, 230 m² Paris 1",Paris 1,333 000 €,Splendide maison en pierres 230 mètres carrés...,IMMOBILIERE DE LA REPUBLIQUE,Maison,
32,32,"Appartement 1 pièce, 36 m² Paris 17",Paris 17,499 000 €,"VILLIERS 6 17è quartier Villiers, beau studio...",PIERRE DE TAILLE CONSEIL IMMOBILIER,Appartement,
...,...,...,...,...,...,...,...,...
17423,17423,"Appartement 1 pièce, 21 m² Paris 8",Paris 8,578 000 €,Vente Appartement 1 pièce IAD France - Gaelle...,iad France,Appartement,
17484,17484,"Appartement 4 pièces, 76 m² Paris 13",Paris 13,937 300 €,"Dpt Paris (75), à vendre PARIS 13EME ARRONDIS...",Capifrance,Appartement,
17498,17498,Appartement 4 pièces Paris 17,Paris 17,980 000 €,"75017, rénové et prêt à emménager, bel appart...",LEGGETT IMMOBILIER,Appartement,
17539,17539,"Appartement 3 pièces, 80 m² Paris 18",Paris 18,965 000 €,Appartement Paris 3 pièce(s) 64 m2 Retrouvez ...,PIERRE INVEST,Appartement,


In [430]:
df.ville_cp=df.ville_cp.astype('str')

In [431]:
#df.ville_cp = df.ville_cp.apply(lambda x: 'nc' if x=='Immobilier  / Vente immobilier ' else x)
        

In [432]:
#df[df.ville_cp.isin(['nc'])]

In [433]:
#df_erreurs_cp =df[df.code_postal == '']
#df_erreurs_cp.to_csv('pv_paritculierIdf.csv')

In [434]:
df[df.ville_cp=='']

,Unnamed: 0,titre_annonce,ville_cp,prix,texte_annonce,nom_agence,type_bien,code_postal


### Conversion en 'nc' pour cellules CP marquées '/n'

In [435]:
df[df.ville_cp=='\n']

,Unnamed: 0,titre_annonce,ville_cp,prix,texte_annonce,nom_agence,type_bien,code_postal
1219,1219,"Appartement 5 pièces, 115 m²",\n,385 000 €,LA ROCHETTE - DUPLEX DE 5 PIÈCES DE 114 M2 AV...,2M IMMOBILIER,Appartement,
1221,1221,"Maison 6 pièces, 130 m²",\n,305 225 €,A VENDRE MAISON MELUN GARE 6 PIECES 130 m2 MA...,2M IMMOBILIER,Maison,
2402,2402,"Maison 7 pièces, 183 m²",\n,288 000 €,"Provins, maison en pierres, excellent état, 7...",LMD IMMOBILIER,Maison,
2784,2784,"Appartement 4 pièces, 89 m²",\n,385 000 €,"4 pièces de 89m2 A proximité du centre ville,...",COTE PARTICULIERS CHATILLON,Appartement,
3392,3392,"Maison 9 pièces, 185 m²",\n,595 000 €,Melun SURPRENANT!!! Dans le centre ville vous...,2M IMMOBILIER,Maison,
3586,3586,"Appartement 4 pièces, 86 m²",\n,562 000 €,appartement 4 pièces aux sablons A 300 mètres...,COTE PARTICULIERS CHATILLON,Appartement,
3598,3598,"Maison 5 pièces, 160 m²",\n,1 220 000 €,Villa a proximité du Métro Dans une impasse p...,COTE PARTICULIERS CHATILLON,Maison,
3658,3658,"Maison 4 pièces, 120 m²",\n,270 000 €,"Maison Meulière, secteur calme, Dammarie Les ...",2M IMMOBILIER,Maison,
3832,3832,"Maison 10 pièces, 250 m²",\n,540 000 €,A VENDRE CORPS DE FERME EN BON ETAT A FORT PO...,2M IMMOBILIER,Maison,
3849,3849,"Appartement 5 pièces, 155 m²",\n,4 500 000 €,PARIS 14e: Rue campagne première L'agence TRI...,TRIOTIM,Appartement,


In [436]:
df.ville_cp = df.ville_cp.apply(lambda x: 'nc' if x=='\n' else x)

In [437]:
df[df.ville_cp == 'nc']

,Unnamed: 0,titre_annonce,ville_cp,prix,texte_annonce,nom_agence,type_bien,code_postal
1219,1219,"Appartement 5 pièces, 115 m²",nc,385 000 €,LA ROCHETTE - DUPLEX DE 5 PIÈCES DE 114 M2 AV...,2M IMMOBILIER,Appartement,
1221,1221,"Maison 6 pièces, 130 m²",nc,305 225 €,A VENDRE MAISON MELUN GARE 6 PIECES 130 m2 MA...,2M IMMOBILIER,Maison,
2402,2402,"Maison 7 pièces, 183 m²",nc,288 000 €,"Provins, maison en pierres, excellent état, 7...",LMD IMMOBILIER,Maison,
2784,2784,"Appartement 4 pièces, 89 m²",nc,385 000 €,"4 pièces de 89m2 A proximité du centre ville,...",COTE PARTICULIERS CHATILLON,Appartement,
3392,3392,"Maison 9 pièces, 185 m²",nc,595 000 €,Melun SURPRENANT!!! Dans le centre ville vous...,2M IMMOBILIER,Maison,
3586,3586,"Appartement 4 pièces, 86 m²",nc,562 000 €,appartement 4 pièces aux sablons A 300 mètres...,COTE PARTICULIERS CHATILLON,Appartement,
3598,3598,"Maison 5 pièces, 160 m²",nc,1 220 000 €,Villa a proximité du Métro Dans une impasse p...,COTE PARTICULIERS CHATILLON,Maison,
3658,3658,"Maison 4 pièces, 120 m²",nc,270 000 €,"Maison Meulière, secteur calme, Dammarie Les ...",2M IMMOBILIER,Maison,
3832,3832,"Maison 10 pièces, 250 m²",nc,540 000 €,A VENDRE CORPS DE FERME EN BON ETAT A FORT PO...,2M IMMOBILIER,Maison,
3849,3849,"Appartement 5 pièces, 155 m²",nc,4 500 000 €,PARIS 14e: Rue campagne première L'agence TRI...,TRIOTIM,Appartement,


### Pour paris: extraction arrondissement dans colonne ville_cp et insertion dans colonne CP avec formatage au format code postal

In [438]:
for i in range(len(df.index)):
        if 'Paris'in df.iloc[i, 2]:        
            df.iloc[i, 7] = '750'+re.sub(r'Paris\s', '',df.iloc[i, 2])
        


In [439]:
df[df.code_postal=='']

,Unnamed: 0,titre_annonce,ville_cp,prix,texte_annonce,nom_agence,type_bien,code_postal
1219,1219,"Appartement 5 pièces, 115 m²",nc,385 000 €,LA ROCHETTE - DUPLEX DE 5 PIÈCES DE 114 M2 AV...,2M IMMOBILIER,Appartement,
1221,1221,"Maison 6 pièces, 130 m²",nc,305 225 €,A VENDRE MAISON MELUN GARE 6 PIECES 130 m2 MA...,2M IMMOBILIER,Maison,
2402,2402,"Maison 7 pièces, 183 m²",nc,288 000 €,"Provins, maison en pierres, excellent état, 7...",LMD IMMOBILIER,Maison,
2784,2784,"Appartement 4 pièces, 89 m²",nc,385 000 €,"4 pièces de 89m2 A proximité du centre ville,...",COTE PARTICULIERS CHATILLON,Appartement,
3392,3392,"Maison 9 pièces, 185 m²",nc,595 000 €,Melun SURPRENANT!!! Dans le centre ville vous...,2M IMMOBILIER,Maison,
3586,3586,"Appartement 4 pièces, 86 m²",nc,562 000 €,appartement 4 pièces aux sablons A 300 mètres...,COTE PARTICULIERS CHATILLON,Appartement,
3598,3598,"Maison 5 pièces, 160 m²",nc,1 220 000 €,Villa a proximité du Métro Dans une impasse p...,COTE PARTICULIERS CHATILLON,Maison,
3658,3658,"Maison 4 pièces, 120 m²",nc,270 000 €,"Maison Meulière, secteur calme, Dammarie Les ...",2M IMMOBILIER,Maison,
3832,3832,"Maison 10 pièces, 250 m²",nc,540 000 €,A VENDRE CORPS DE FERME EN BON ETAT A FORT PO...,2M IMMOBILIER,Maison,
3849,3849,"Appartement 5 pièces, 155 m²",nc,4 500 000 €,PARIS 14e: Rue campagne première L'agence TRI...,TRIOTIM,Appartement,


In [440]:
# na dans code_postal ramplacé par nc
df.code_postal=df.code_postal.apply(lambda x: 'nc' if x == '' else x)

In [441]:
# retrait code postal dans ville 
df.ville_cp = df.ville_cp.apply(lambda x: re.sub(r'\((\w){5}\)', '', x))

In [442]:
# renommer cp_ville en ville
df.rename(columns={'ville_cp':'ville'}, inplace= True)

In [443]:
# CONVERSION DES CP PARISIENS  À 4 CHIFFRES EN 5 CHIFFRES (7501 TO 75001)
# création d'une liste avec 2 sous listes des codes postaux au mauvais format et l'autre au bon format
# (7501 et 75001 par ex)

cp_g = [list(range(7501, 7510)), list(range(75001, 75010))]
print(cp_g)

[[7501, 7502, 7503, 7504, 7505, 7506, 7507, 7508, 7509], [75001, 75002, 75003, 75004, 75005, 75006, 75007, 75008, 75009]]


In [444]:
# boucle pour chercher cp parisiens au mavais format et les convertir au bon format en iterant dans la 2eme 
# sous liste de cp_g
for i in range(len(df.index)):
    for l in range(len(cp_g[0])):        
        if df.iloc[i, 7] in str(cp_g[0][l]): #attention penser à mettre str car liste crée = digits et cp = str
            df.iloc[i, 7] = str(cp_g[1][l])
        else:
            pass


In [445]:
# corrections erreurs dans cp
df[df.code_postal=='750(75116)']
df.code_postal = df.code_postal.apply(lambda x: '75016' if x=="750(75116)" else x)

In [446]:
df.code_postal = df.code_postal.apply(lambda x: '95190' if x=='750Fontenay-en-Parisis (95190)' else x)

In [447]:
df.code_postal = df.code_postal.apply(lambda x: '95240' if x=='750Cormeilles-en-Parisis (95240)' else x)

In [448]:
# vérifications que foramt cp ok
df.code_postal.unique() 

array(['75018', '75008', '91200', '91070', '77260', '75015', '95300',
       '91260', '91670', '77174', '77515', '77140', '91940', '95290',
       '95260', '77186', '77500', '77340', '77270', '95340', '95590',
       '75001', '91590', '93300', '75017', '91820', '77510', '75010',
       '78610', '92700', '91150', '93390', '94120', '95100', '92140',
       '93470', '93110', '94320', '95400', '78150', '78390', '91540',
       '77120', '93150', '77320', '78980', '78600', '77118', '94140',
       '77710', '95480', '78700', '91160', '78200', '78260', '91380',
       '77480', '92390', '78000', '75013', '77160', '93330', '78440',
       '78190', '77420', '93370', '77000', '77670', '77167', '77250',
       '77130', '78550', '95200', '77210', '95210', '91600', '93420',
       '94600', '95320', '78380', '78730', '92410', '91760', '77114',
       '77650', '78930', '93250', '93220', '77700', '75011', '93700',
       '93350', '94380', '77190', '91120', '77144', '91170', '77400',
       '78955', '785

In [449]:
#df[df.code_postal == '750Paris']

### Colonne ville : retrait arrondissement après ville Paris

In [450]:
#df.ville=df.ville.astype('str')
for i in range(len(df.index)):
    if 'Paris ' in df.iloc[i, 2]:
        df.iloc[i, 2] = 'Paris'
    else:
        pass

In [451]:
df[df.ville == 'Paris']

,Unnamed: 0,titre_annonce,ville,prix,texte_annonce,nom_agence,type_bien,code_postal
0,0,"Appartement 2 pièces, 33 m² Paris 18",Paris,420 000 €,Place Saint Pierre / Butte Montmartrre. 2 piè...,DISTRICT GEORGE V,Appartement,75018
1,1,"Appartement 5 pièces, 146 m² Paris 8",Paris,2 660 000 €,TRIANGLE d'OR proche de l'avenue Montaigne - ...,DISTRICT GEORGE V,Appartement,75008
5,5,"Appartement 2 pièces, 47 m² Paris 15",Paris,415 000 €,Appartement 2 pièces 47m2 à Paris en résidenc...,EXPERTIMO,Appartement,75015
27,27,"Maison 9 pièces, 230 m² Paris 1",Paris,333 000 €,Splendide maison en pierres 230 mètres carrés...,IMMOBILIERE DE LA REPUBLIQUE,Maison,75001
32,32,"Appartement 1 pièce, 36 m² Paris 17",Paris,499 000 €,"VILLIERS 6 17è quartier Villiers, beau studio...",PIERRE DE TAILLE CONSEIL IMMOBILIER,Appartement,75017
...,...,...,...,...,...,...,...,...
17423,17423,"Appartement 1 pièce, 21 m² Paris 8",Paris,578 000 €,Vente Appartement 1 pièce IAD France - Gaelle...,iad France,Appartement,75008
17484,17484,"Appartement 4 pièces, 76 m² Paris 13",Paris,937 300 €,"Dpt Paris (75), à vendre PARIS 13EME ARRONDIS...",Capifrance,Appartement,75013
17498,17498,Appartement 4 pièces Paris 17,Paris,980 000 €,"75017, rénové et prêt à emménager, bel appart...",LEGGETT IMMOBILIER,Appartement,75017
17539,17539,"Appartement 3 pièces, 80 m² Paris 18",Paris,965 000 €,Appartement Paris 3 pièce(s) 64 m2 Retrouvez ...,PIERRE INVEST,Appartement,75018


# Formatage colonne 'prix'

In [452]:
#retrait sigle euro dans prix
df.prix = df.prix.apply(lambda x: re.sub(r'\s€', '', x))
#retrait espaces dans prix
df.prix = df.prix.apply(lambda x: re.sub(r'\s', '', x))

In [453]:
#retrait des lignes ou prix = NC (une dizaine de lignes)
df = df[df.prix!='NC']
df[df.prix=='NC']

,Unnamed: 0,titre_annonce,ville,prix,texte_annonce,nom_agence,type_bien,code_postal


In [454]:
# conversion colonne prix en valeur numérique
df.prix = df.prix.apply(lambda x: int(x))

In [455]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16496 entries, 0 to 17599
Data columns (total 8 columns):
Unnamed: 0       16496 non-null int64
titre_annonce    16496 non-null object
ville            16496 non-null object
prix             16496 non-null int64
texte_annonce    16496 non-null object
nom_agence       16496 non-null object
type_bien        16496 non-null object
code_postal      16496 non-null object
dtypes: int64(2), object(6)
memory usage: 1.1+ MB


# Création variable 'Surface'

In [456]:
#création colonne surface
df['surface'] = ''

In [457]:
df.head()

,Unnamed: 0,titre_annonce,ville,prix,texte_annonce,nom_agence,type_bien,code_postal,surface
0,0,"Appartement 2 pièces, 33 m² Paris 18",Paris,420000,Place Saint Pierre / Butte Montmartrre. 2 piè...,DISTRICT GEORGE V,Appartement,75018,
1,1,"Appartement 5 pièces, 146 m² Paris 8",Paris,2660000,TRIANGLE d'OR proche de l'avenue Montaigne - ...,DISTRICT GEORGE V,Appartement,75008,
2,2,"Maison 5 pièces, 65 m² Athis-Mons (91200)",Athis-Mons,334000,PLATEAU JOLIE MEULIERE 120 M² CALME NON MITOY...,Réseau Swixim,Maison,91200,
3,3,"Appartement 3 pièces, 70 m² carrez Bondoufle ...",Bondoufle,202800,VIEUX CENTRE - Appartement 3 pièces - 70m² Si...,,Appartement,91070,
4,4,"Maison 3 pièces, 66 m² La Ferté-sous-Jouarre ...",La Ferté-sous-Jouarre,95500,"Maison de hameau avec courette, vue dégagée A...",AGENCE DE LA VALLEE,Maison,77260,


### Extraction surface de la colonne titre annonce et insertion dans colonne surface

In [458]:
surface_reg = re.compile(r'\w+\sm²') # pattern surface dans titre_annnoce

    
for i in range(len(df.index)): # insertion surface dans colonne surface
    try: # pour passer outre les valaurs nulles ou absence de surface dans annonce
        df.iloc[i, 8] = surface_reg.search(df.iloc[i, 1]).group()
    except:
        pass
    else:
        df.iloc[i, 8] = surface_reg.search(df.iloc[i, 1]).group()

In [459]:
df.head()

,Unnamed: 0,titre_annonce,ville,prix,texte_annonce,nom_agence,type_bien,code_postal,surface
0,0,"Appartement 2 pièces, 33 m² Paris 18",Paris,420000,Place Saint Pierre / Butte Montmartrre. 2 piè...,DISTRICT GEORGE V,Appartement,75018,33 m²
1,1,"Appartement 5 pièces, 146 m² Paris 8",Paris,2660000,TRIANGLE d'OR proche de l'avenue Montaigne - ...,DISTRICT GEORGE V,Appartement,75008,146 m²
2,2,"Maison 5 pièces, 65 m² Athis-Mons (91200)",Athis-Mons,334000,PLATEAU JOLIE MEULIERE 120 M² CALME NON MITOY...,Réseau Swixim,Maison,91200,65 m²
3,3,"Appartement 3 pièces, 70 m² carrez Bondoufle ...",Bondoufle,202800,VIEUX CENTRE - Appartement 3 pièces - 70m² Si...,,Appartement,91070,70 m²
4,4,"Maison 3 pièces, 66 m² La Ferté-sous-Jouarre ...",La Ferté-sous-Jouarre,95500,"Maison de hameau avec courette, vue dégagée A...",AGENCE DE LA VALLEE,Maison,77260,66 m²


In [460]:
# insertion 'nc' pour valeurs nulles dans surface
df.surface = df.surface.apply(lambda x: 'nc' if x == '' else x)

In [461]:
df[df.surface=='']

,Unnamed: 0,titre_annonce,ville,prix,texte_annonce,nom_agence,type_bien,code_postal,surface


### Suppression de 'm2' dans surface afin d'avoir uniquement une valeur numérique

In [462]:
df.surface = df.surface.apply(lambda x : re.sub(r'\sm²', '', x))

### Remplacement par 'nc' dans surface quand 1m2 est indiqué

In [477]:
df.surface = df.surface.apply(lambda x: 'nc' if x==None else x)

In [479]:

df.surface = df.surface.apply(lambda x: None if x=='nc' else x)

#### df.surface=df.surface.astype('Int64')

In [482]:
df.surface = df.surface.apply(lambda x: int(x) if x!=None else x)

In [483]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16496 entries, 0 to 17599
Data columns (total 9 columns):
Unnamed: 0       16496 non-null int64
titre_annonce    16496 non-null object
ville            16496 non-null object
prix             16496 non-null int64
texte_annonce    16496 non-null object
nom_agence       16496 non-null object
type_bien        16496 non-null object
code_postal      16496 non-null object
surface          16427 non-null float64
dtypes: float64(1), int64(2), object(6)
memory usage: 1.9+ MB


# Suppresion double colonne d'index (j'aurais sû le faire à l'import en précisant paramètre spécifique dans pd.csv_read())

In [484]:
# suppression double colonne d'index. Je n'en garde qu'une
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [485]:
# création colonne prix au m2
df['prix_m2'] = ''

In [284]:
#conversion colonne surface en int pour calcul prix/m2
#df['surface'] = df['surface'].apply(lambda x: int(x) if x != 'nc' else x)

In [486]:
# calcul prix au mètre carré arrondi
for i in range(len(df.index)):
    if df.iloc[i, 7] != 'nc':
        df.iloc[i, 8] = round((df.iloc[i, 2] / df.iloc[i, 7]), 0)
    else:
        df.iloc[i, 8] = None

In [487]:
df.head()

,titre_annonce,ville,prix,texte_annonce,nom_agence,type_bien,code_postal,surface,prix_m2
0,"Appartement 2 pièces, 33 m² Paris 18",Paris,420000,Place Saint Pierre / Butte Montmartrre. 2 piè...,DISTRICT GEORGE V,Appartement,75018,33.0,12727
1,"Appartement 5 pièces, 146 m² Paris 8",Paris,2660000,TRIANGLE d'OR proche de l'avenue Montaigne - ...,DISTRICT GEORGE V,Appartement,75008,146.0,18219
2,"Maison 5 pièces, 65 m² Athis-Mons (91200)",Athis-Mons,334000,PLATEAU JOLIE MEULIERE 120 M² CALME NON MITOY...,Réseau Swixim,Maison,91200,65.0,5138
3,"Appartement 3 pièces, 70 m² carrez Bondoufle ...",Bondoufle,202800,VIEUX CENTRE - Appartement 3 pièces - 70m² Si...,,Appartement,91070,70.0,2897
4,"Maison 3 pièces, 66 m² La Ferté-sous-Jouarre ...",La Ferté-sous-Jouarre,95500,"Maison de hameau avec courette, vue dégagée A...",AGENCE DE LA VALLEE,Maison,77260,66.0,1447


In [290]:
df.prix_m2 = df.prix_m2.apply(lambda x: int(x) if x != None else x)

In [506]:
df[df.surface<=6]

,titre_annonce,ville,prix,texte_annonce,nom_agence,type_bien,code_postal,surface,prix_m2,nbre_pieces
824,"Appartement 1 pièce, 6 m² Paris 10",Paris,70000,"Avec vue sur les toits et la Tour Eiffel, cha...",L'ADRESSE DU CANAL,Appartement,75010,6.0,11667,1
5600,"Appartement 1 pièce, 6 m² Paris 11",Paris,65000,PARIS 11ème -1 MINUTE METRO VOLTAIRE- Au 6 èm...,L'ADRESSE OBERKAMPF,Appartement,75011,6.0,10833,1
8154,"Appartement 1 pièce, 6 m² Paris 14",Paris,89500,"Dpt Paris (75), à vendre PARIS 14EME ARRONDIS...",Capifrance,Appartement,75014,6.0,14917,1
9752,"Appartement 1 pièce, 6 m² Saint-Ouen (93400)",Saint-Ouen,49000,Vente Appartement 1 pièce iad France - Marie-...,iad France,Appartement,93400,6.0,8167,1
12898,"Appartement 1 pièce, 6 m² Paris 8",Paris,88000,Chambre de bonne Triangle d'OR Félix MAHON de...,Dr House Immo,Appartement,75008,6.0,14667,1


In [507]:
# valeur 1 et 2 converti en None
df.surface=df.surface.apply(lambda x: None if x == 1 or x == 2 else x)


In [508]:
df[df.surface<=6]

,titre_annonce,ville,prix,texte_annonce,nom_agence,type_bien,code_postal,surface,prix_m2,nbre_pieces
824,"Appartement 1 pièce, 6 m² Paris 10",Paris,70000,"Avec vue sur les toits et la Tour Eiffel, cha...",L'ADRESSE DU CANAL,Appartement,75010,6.0,11667,1
5600,"Appartement 1 pièce, 6 m² Paris 11",Paris,65000,PARIS 11ème -1 MINUTE METRO VOLTAIRE- Au 6 èm...,L'ADRESSE OBERKAMPF,Appartement,75011,6.0,10833,1
8154,"Appartement 1 pièce, 6 m² Paris 14",Paris,89500,"Dpt Paris (75), à vendre PARIS 14EME ARRONDIS...",Capifrance,Appartement,75014,6.0,14917,1
9752,"Appartement 1 pièce, 6 m² Saint-Ouen (93400)",Saint-Ouen,49000,Vente Appartement 1 pièce iad France - Marie-...,iad France,Appartement,93400,6.0,8167,1
12898,"Appartement 1 pièce, 6 m² Paris 8",Paris,88000,Chambre de bonne Triangle d'OR Félix MAHON de...,Dr House Immo,Appartement,75008,6.0,14667,1


## CRÉATION VARIABLE NOMBRE DE PIÈCES ET EXTRACTION DE L'INFO DANS COLONNE TITRE_ANNONCE

In [499]:
# creation variable pieces pattern opur récupérer nbre de pieces dans colonne titre_annonce
pieces_reg = re.compile(r'\w*\spièce(s)?')
df['nbre_pieces'] = None
df.head()

,titre_annonce,ville,prix,texte_annonce,nom_agence,type_bien,code_postal,surface,prix_m2,nbre_pieces
0,"Appartement 2 pièces, 33 m² Paris 18",Paris,420000,Place Saint Pierre / Butte Montmartrre. 2 piè...,DISTRICT GEORGE V,Appartement,75018,33.0,12727,None
1,"Appartement 5 pièces, 146 m² Paris 8",Paris,2660000,TRIANGLE d'OR proche de l'avenue Montaigne - ...,DISTRICT GEORGE V,Appartement,75008,146.0,18219,None
2,"Maison 5 pièces, 65 m² Athis-Mons (91200)",Athis-Mons,334000,PLATEAU JOLIE MEULIERE 120 M² CALME NON MITOY...,Réseau Swixim,Maison,91200,65.0,5138,None
3,"Appartement 3 pièces, 70 m² carrez Bondoufle ...",Bondoufle,202800,VIEUX CENTRE - Appartement 3 pièces - 70m² Si...,,Appartement,91070,70.0,2897,None
4,"Maison 3 pièces, 66 m² La Ferté-sous-Jouarre ...",La Ferté-sous-Jouarre,95500,"Maison de hameau avec courette, vue dégagée A...",AGENCE DE LA VALLEE,Maison,77260,66.0,1447,None


In [500]:
# extration nombre de pieces de titre annonces vers colonne nbre de pieces
for i in range(len(df.index)):
    try:
        df.iloc[i, 9] = int(re.sub(r'\spièce(s)?', '',pieces_reg.search(df.iloc[i,0]).group()))
    except:
        pass
    else:
        df.iloc[i, 9] = int(re.sub(r'\spièce(s)?', '',pieces_reg.search(df.iloc[i,0]).group()))

In [511]:
df['type_annonce'] = 'Professionnel'
df.head()

,titre_annonce,ville,prix,texte_annonce,nom_agence,type_bien,code_postal,surface,prix_m2,nbre_pieces,type_annonce
0,"Appartement 2 pièces, 33 m² Paris 18",Paris,420000,Place Saint Pierre / Butte Montmartrre. 2 piè...,DISTRICT GEORGE V,Appartement,75018,33.0,12727,2,Professionnel
1,"Appartement 5 pièces, 146 m² Paris 8",Paris,2660000,TRIANGLE d'OR proche de l'avenue Montaigne - ...,DISTRICT GEORGE V,Appartement,75008,146.0,18219,5,Professionnel
2,"Maison 5 pièces, 65 m² Athis-Mons (91200)",Athis-Mons,334000,PLATEAU JOLIE MEULIERE 120 M² CALME NON MITOY...,Réseau Swixim,Maison,91200,65.0,5138,5,Professionnel
3,"Appartement 3 pièces, 70 m² carrez Bondoufle ...",Bondoufle,202800,VIEUX CENTRE - Appartement 3 pièces - 70m² Si...,,Appartement,91070,70.0,2897,3,Professionnel
4,"Maison 3 pièces, 66 m² La Ferté-sous-Jouarre ...",La Ferté-sous-Jouarre,95500,"Maison de hameau avec courette, vue dégagée A...",AGENCE DE LA VALLEE,Maison,77260,66.0,1447,3,Professionnel


In [502]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16496 entries, 0 to 17599
Data columns (total 10 columns):
titre_annonce    16496 non-null object
ville            16496 non-null object
prix             16496 non-null int64
texte_annonce    16496 non-null object
nom_agence       16496 non-null object
type_bien        16496 non-null object
code_postal      16496 non-null object
surface          16411 non-null float64
prix_m2          16427 non-null object
nbre_pieces      16278 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 2.0+ MB


In [519]:
df.rename(columns={'nom_agence':'nom_vendeur'}, inplace=True)
df.head()

,titre_annonce,ville,prix,texte_annonce,nom_vendeur,type_bien,code_postal,surface,prix_m2,nbre_pieces,type_annonce
0,"Appartement 2 pièces, 33 m² Paris 18",Paris,420000,Place Saint Pierre / Butte Montmartrre. 2 piè...,DISTRICT GEORGE V,Appartement,75018,33.0,12727,2,Professionnel
1,"Appartement 5 pièces, 146 m² Paris 8",Paris,2660000,TRIANGLE d'OR proche de l'avenue Montaigne - ...,DISTRICT GEORGE V,Appartement,75008,146.0,18219,5,Professionnel
2,"Maison 5 pièces, 65 m² Athis-Mons (91200)",Athis-Mons,334000,PLATEAU JOLIE MEULIERE 120 M² CALME NON MITOY...,Réseau Swixim,Maison,91200,65.0,5138,5,Professionnel
3,"Appartement 3 pièces, 70 m² carrez Bondoufle ...",Bondoufle,202800,VIEUX CENTRE - Appartement 3 pièces - 70m² Si...,,Appartement,91070,70.0,2897,3,Professionnel
4,"Maison 3 pièces, 66 m² La Ferté-sous-Jouarre ...",La Ferté-sous-Jouarre,95500,"Maison de hameau avec courette, vue dégagée A...",AGENCE DE LA VALLEE,Maison,77260,66.0,1447,3,Professionnel


In [524]:
df4=df[['titre_annonce', 'texte_annonce', 'type_bien', 'surface', 'nbre_pieces', 'ville', 'code_postal', 'prix', 'nom_vendeur', 'type_annonce']]



In [525]:
df4.head()

,titre_annonce,texte_annonce,type_bien,surface,nbre_pieces,ville,code_postal,prix,nom_vendeur,type_annonce
0,"Appartement 2 pièces, 33 m² Paris 18",Place Saint Pierre / Butte Montmartrre. 2 piè...,Appartement,33.0,2,Paris,75018,420000,DISTRICT GEORGE V,Professionnel
1,"Appartement 5 pièces, 146 m² Paris 8",TRIANGLE d'OR proche de l'avenue Montaigne - ...,Appartement,146.0,5,Paris,75008,2660000,DISTRICT GEORGE V,Professionnel
2,"Maison 5 pièces, 65 m² Athis-Mons (91200)",PLATEAU JOLIE MEULIERE 120 M² CALME NON MITOY...,Maison,65.0,5,Athis-Mons,91200,334000,Réseau Swixim,Professionnel
3,"Appartement 3 pièces, 70 m² carrez Bondoufle ...",VIEUX CENTRE - Appartement 3 pièces - 70m² Si...,Appartement,70.0,3,Bondoufle,91070,202800,,Professionnel
4,"Maison 3 pièces, 66 m² La Ferté-sous-Jouarre ...","Maison de hameau avec courette, vue dégagée A...",Maison,66.0,3,La Ferté-sous-Jouarre,77260,95500,AGENCE DE LA VALLEE,Professionnel


In [526]:
df4.to_csv('paruvendu_ancien_pro_idf_clean.csv')